<a href="https://colab.research.google.com/github/MeenatchiKV/My-ML-Learnings/blob/master/FeedForward_TF_Iris_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IRIS

In [ ]:
import tensorflow as tf
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

RANDOM_SEED = 42
tf.set_random_seed(RANDOM_SEED)


def init_weights(shape):
    """ Weight initialization """
    weights = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(weights)

def forwardprop(X, w_1, w_2):
    """
    Forward-propagation.
    IMPORTANT: yhat is not softmax since TensorFlow's softmax_cross_entropy_with_logits() does that internally.
    """
    h    = tf.nn.sigmoid(tf.matmul(X, w_1))  # The \sigma function
    yhat = tf.matmul(h, w_2)  # The \varphi function
    return yhat

def get_iris_data():
    """ Read the iris data set and split them into training and test sets """
    iris   = datasets.load_iris()
    data   = iris["data"]
    target = iris["target"]

    # Prepend the column of 1s for bias
    N, M  = data.shape
    all_X = np.ones((N, M + 1))
    all_X[:, 1:] = data

    # Convert into one-hot vectors
    num_labels = len(np.unique(target))
    all_Y = np.eye(num_labels)[target]  # One liner trick!
    return train_test_split(all_X, all_Y, test_size=0.33, random_state=RANDOM_SEED)

def main():
    train_X, test_X, train_y, test_y = get_iris_data()

    # Layer's sizes
    x_size = train_X.shape[1]   # Number of input nodes: 4 features and 1 bias
    h_size = 256                # Number of hidden nodes
    y_size = train_y.shape[1]   # Number of outcomes (3 iris flowers)

    # Symbols
    X = tf.placeholder("float", shape=[None, x_size])
    y = tf.placeholder("float", shape=[None, y_size])

    # Weight initializations
    w_1 = init_weights((x_size, h_size))
    w_2 = init_weights((h_size, y_size))

    # Forward propagation
    yhat    = forwardprop(X, w_1, w_2)
    predict = tf.argmax(yhat, axis=1)

    # Backward propagation
    cost    = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
    updates = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

    # Run SGD
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    for epoch in range(100):
        # Train with each example
        for i in range(len(train_X)):
            sess.run(updates, feed_dict={X: train_X[i: i + 1], y: train_y[i: i + 1]})

        train_accuracy = np.mean(np.argmax(train_y, axis=1) ==
                                 sess.run(predict, feed_dict={X: train_X, y: train_y}))
        test_accuracy  = np.mean(np.argmax(test_y, axis=1) ==
                                 sess.run(predict, feed_dict={X: test_X, y: test_y}))

        print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%"
              % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))

    sess.close()

if __name__ == '__main__':
    main()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch = 1, train accuracy = 40.00%, test accuracy = 38.00%
Epoch = 2, train accuracy = 65.00%, test accuracy = 70.00%
Epoch = 3, train accuracy = 65.00%, test accuracy = 70.00%
Epoch = 4, train accuracy = 66.00%, test accuracy = 70.00%
Epoch = 5, train accuracy = 70.00%, test accuracy = 70.00%
Epoch = 6, train accuracy = 71.00%, test accuracy = 74.00%
Epoch = 7, train accuracy = 72.00%, test accuracy = 78.00%
Epoch = 8, train accuracy = 77.00%, test accuracy = 78.00%
Epoch = 9, train accuracy = 79.00%, test accuracy = 78.00%
Epoch = 10, train accuracy = 82.00%, test accuracy = 78.00%
Epoch = 11, train accuracy = 83.00%, test accuracy = 78.00%
Epoch = 12, train accuracy = 87.00%, test accuracy = 78.00%
Epoch = 13, train accuracy = 88.00%, test accuracy = 78.00%
Epoch = 14, train accuracy = 88.00%, t

# XOR

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

sess = tf.InteractiveSession()

# a batch of inputs of 2 value each
inputs = tf.placeholder(tf.float32, shape=[None, 2])

# a batch of output of 1 value each
desired_outputs = tf.placeholder(tf.float32, shape=[None, 1])

# [!] define the number of hidden units in the first layer
HIDDEN_UNITS = 4 

# connect 2 inputs to 3 hidden units
# [!] Initialize weights with random numbers, to make the network learn
weights_1 = tf.Variable(tf.truncated_normal([2, HIDDEN_UNITS]))

# [!] The biases are single values per hidden unit
biases_1 = tf.Variable(tf.zeros([HIDDEN_UNITS]))

# connect 2 inputs to every hidden unit. Add bias
layer_1_outputs = tf.nn.sigmoid(tf.matmul(inputs, weights_1) + biases_1)

# [!] The XOR problem is that the function is not linearly separable
# [!] A MLP (Multi layer perceptron) can learn to separe non linearly separable points ( you can
# think that it will learn hypercurves, not only hyperplanes)
# [!] Lets' add a new layer and change the layer 2 to output more than 1 value

# connect first hidden units to 2 hidden units in the second hidden layer
weights_2 = tf.Variable(tf.truncated_normal([HIDDEN_UNITS, 2]))
# [!] The same of above
biases_2 = tf.Variable(tf.zeros([2]))

# connect the hidden units to the second hidden layer
layer_2_outputs = tf.nn.sigmoid(
    tf.matmul(layer_1_outputs, weights_2) + biases_2)

# [!] create the new layer
weights_3 = tf.Variable(tf.truncated_normal([2, 1]))
biases_3 = tf.Variable(tf.zeros([1]))

logits = tf.nn.sigmoid(tf.matmul(layer_2_outputs, weights_3) + biases_3)

# [!] The error function chosen is good for a multiclass classification taks, not for a XOR.
error_function = 0.5 * tf.reduce_sum(tf.subtract(logits, desired_outputs) * tf.subtract(logits, desired_outputs))

train_step = tf.train.GradientDescentOptimizer(0.05).minimize(error_function)

sess.run(tf.initialize_all_variables())

training_inputs = [[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]]

training_outputs = [[0.0], [1.0], [1.0], [0.0]]

for i in range(20000):
    _, loss = sess.run([train_step, error_function],
                       feed_dict={inputs: np.array(training_inputs),
                                  desired_outputs: np.array(training_outputs)})
    print(loss)

print(sess.run(logits, feed_dict={inputs: np.array([[0.0, 0.0]])}))
print(sess.run(logits, feed_dict={inputs: np.array([[0.0, 1.0]])}))
print(sess.run(logits, feed_dict={inputs: np.array([[1.0, 0.0]])}))
print(sess.run(logits, feed_dict={inputs: np.array([[1.0, 1.0]])}))

c:\users\sandhya\appdata\local\programs\python\python36\lib\site-packages\tensorflow_core\python\client\session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Instructions for updating:
Use `tf.global_variables_initializer` instead.
0.53714657
0.53537667
0.5336801
0.5320548
0.53049886
0.52901006
0.5275867
0.52622634
0.524927
0.52368677
0.5225034
0.5213747
0.5202989
0.51927376
0.51829743
0.51736796
0.5164833
0.5156418
0.51484144
0.5140805
0.5133573
0.5126702
0.5120175
0.5113978
0.51080936
0.51025087
0.5097209
0.5092181
0.50874126
0.5082889
0.5078601
0.50745356
0.5070683
0.5067031
0.5063571
0.5060292
0.5057187
0.5054245
0.5051458
0.504882
0.5046322
0.5043956
0.50417167
0.5039598
0.503759
0.5035692
0.5033895
0.50321937
0.50305843
0.50290614
0.5027621
0.50262576
0.5024967
0.50237465
0.50225925
0.50215006
0.50204676
0.501949
0.50185657
0.5017692
0.5016866
0.50160843
0.5015344
0.50146437
0.5013983
0.5013358
0.50127655
0.5012206
0.50116765
0.50111765
0.5010704
0.50102556
0.50098324
0.50094324
0.50090533
0.5008695
0.5008357
0.5008037
0.50077343
0.5007448
0.50071764
0.5006921
0.5006679
0.50064504
0.5006234
0.5006029
0.5005835
0.5005652
0.5005479
0.50

0.50012505
0.5001248
0.5001247
0.5001246
0.50012434
0.50012434
0.5001241
0.500124
0.5001238
0.5001236
0.5001235
0.5001234
0.5001232
0.5001231
0.5001229
0.5001227
0.5001226
0.5001225
0.50012237
0.50012225
0.50012195
0.5001219
0.5001217
0.5001216
0.5001214
0.5001213
0.5001212
0.500121
0.5001208
0.50012064
0.5001205
0.5001204
0.5001202
0.5001201
0.50012
0.50011986
0.5001196
0.5001195
0.5001194
0.5001191
0.5001191
0.50011885
0.50011873
0.5001186
0.50011843
0.5001184
0.5001182
0.500118
0.5001179
0.5001178
0.50011754
0.5001174
0.5001173
0.5001172
0.50011694
0.5001168
0.5001167
0.5001165
0.50011647
0.5001162
0.5001161
0.50011593
0.50011575
0.50011563
0.5001155
0.5001154
0.5001153
0.50011504
0.5001149
0.5001148
0.5001146
0.50011444
0.5001143
0.50011414
0.500114
0.50011384
0.5001138
0.50011355
0.5001134
0.5001133
0.5001131
0.500113
0.5001129
0.5001128
0.50011253
0.5001124
0.5001123
0.5001121
0.50011206
0.50011176
0.5001117
0.5001115
0.50011134
0.5001112
0.5001111
0.5001109
0.5001108
0.5001106
0

0.49999207
0.49999198
0.49999177
0.4999917
0.4999916
0.49999142
0.49999124
0.49999112
0.499991
0.49999085
0.4999907
0.4999905
0.49999043
0.49999028
0.49999017
0.49999005
0.49998987
0.4999898
0.49998966
0.49998948
0.4999894
0.49998924
0.4999891
0.4999889
0.4999888
0.49998862
0.4999885
0.49998832
0.4999882
0.49998808
0.49998802
0.49998778
0.49998766
0.49998748
0.49998742
0.4999873
0.49998719
0.499987
0.4999869
0.49998677
0.4999866
0.49998647
0.49998635
0.49998623
0.499986
0.49998593
0.49998575
0.49998564
0.49998546
0.49998534
0.49998522
0.4999851
0.49998492
0.4999848
0.49998468
0.49998456
0.49998438
0.49998426
0.49998415
0.49998397
0.4999839
0.49998373
0.49998355
0.49998343
0.4999833
0.4999832
0.49998307
0.49998295
0.49998283
0.49998266
0.49998254
0.49998236
0.49998224
0.49998206
0.499982
0.49998182
0.4999817
0.49998158
0.4999814
0.49998122
0.49998105
0.49998096
0.4999808
0.49998066
0.49998054
0.49998042
0.49998027
0.49998012
0.49997997
0.49997985
0.49997967
0.49997962
0.4999794
0.499979

0.49988467
0.49988455
0.49988443
0.4998843
0.49988413
0.49988395
0.49988383
0.4998837
0.4998836
0.49988347
0.49988335
0.49988317
0.49988306
0.49988288
0.49988276
0.4998827
0.49988252
0.4998824
0.49988222
0.49988204
0.49988198
0.4998818
0.49988168
0.49988157
0.49988145
0.49988127
0.4998812
0.49988103
0.49988085
0.4998808
0.49988067
0.4998805
0.49988037
0.4998802
0.49988008
0.49988
0.49987975
0.49987966
0.49987948
0.4998794
0.4998792
0.49987906
0.49987894
0.49987888
0.4998787
0.49987862
0.4998784
0.49987832
0.49987826
0.49987808
0.49987787
0.49987775
0.4998776
0.4998775
0.49987742
0.49987724
0.49987715
0.49987704
0.4998769
0.49987665
0.49987653
0.4998764
0.4998763
0.49987614
0.49987605
0.49987587
0.49987575
0.49987563
0.4998755
0.49987534
0.49987522
0.49987507
0.49987492
0.4998748
0.49987468
0.49987453
0.4998744
0.4998743
0.49987411
0.499874
0.49987385
0.49987373
0.49987364
0.49987346
0.4998733
0.49987322
0.49987304
0.4998729
0.4998728
0.49987265
0.49987254
0.49987242
0.49987224
0.499872

...

0.3680897
0.367983
0.36787617
0.36776942
0.36766243
0.36755538
0.36744818
0.367341
0.36723363
0.36712623
0.36701864
0.366911
0.3668033
0.36669552
0.36658758
0.36647946
0.36637136
0.36626315
0.36615473
0.36604637
0.36593783
0.36582917
0.36572048
0.36561158
0.3655026
0.36539358
0.3652845
0.3651752
0.36506587
0.3649564
0.36484683
0.3647372
0.36462748
0.36451763
0.3644076
0.3642975
0.36418733
0.36407703
0.36396667
0.3638563
0.36374563
0.36363488
0.36352408
0.3634132
0.36330217
0.3631911
0.36307985
0.36296853
0.362857
0.36274546
0.3626339
0.36252207
0.36241022
0.36229828
0.36218613
0.362074
0.36196172
0.3618493
0.36173674
0.36162412
0.36151144
0.36139867
0.36128563
0.36117256
0.3610595
0.36094615
0.36083275
0.3607192
0.36060566
0.3604919
0.36037815
0.3602642
0.36015016
0.36003593
0.3599217
0.35980722
0.35969275
0.3595782
0.35946342
0.3593486
0.35923362
0.35911858
0.35900342
0.35888812
0.35877267
0.35865715
0.35854155
0.35842574
0.3583099
0.35819396
0.35807776
0.3579616
0.3578453
0.35772887


0.21260369
0.21237239
0.21214107
0.21190973
0.21167842
0.21144712
0.2112158
0.21098457
0.21075322
0.21052197
0.21029074
0.21005954
0.20982835
0.20959714
0.20936596
0.2091348
0.20890364
0.20867258
0.20844153
0.20821036
0.20797932
0.20774835
0.20751737
0.2072864
0.20705545
0.20682456
0.20659357
0.20636277
0.20613192
0.20590106
0.20567037
0.2054396
0.2052089
0.20497821
0.20474768
0.20451705
0.20428652
0.20405596
0.20382552
0.20359504
0.20336469
0.20313433
0.202904
0.20267376
0.20244351
0.20221329
0.2019832
0.20175308
0.201523
0.20129302
0.2010631
0.2008332
0.20060337
0.20037356
0.20014378
0.1999141
0.19968456
0.19945487
0.19922541
0.19899592
0.19876656
0.19853723
0.1983079
0.19807866
0.19784951
0.19762036
0.19739136
0.19716236
0.19693345
0.1967046
0.19647583
0.1962471
0.19601846
0.19578989
0.19556141
0.19533296
0.19510454
0.19487625
0.19464807
0.1944199
0.19419181
0.19396383
0.19373587
0.19350809
0.19328019
0.19305257
0.19282493
0.19259739
0.19237
0.1921426
0.19191533
0.19168815
0.1914610

0.0711654
0.07108001
0.070994854
0.07090979
0.07082483
0.07074007
0.070655435
0.07057091
0.070486546
0.07040234
0.07031824
0.070234284
0.0701505
0.07006686
0.06998331
0.06989992
0.069816634
0.069733545
0.06965056
0.0695677
0.06948498
0.0694024
0.06932
0.069237694
0.06915554
0.06907346
0.06899164
0.068909846
0.068828225
0.06874673
0.06866537
0.06858409
0.06850307
0.068422094
0.06834126
0.06826059
0.06818002
0.06809958
0.06801931
0.06793915
0.06785912
0.06777923
0.06769943
0.067619815
0.06754033
0.06746097
0.0673817
0.067302585
0.06722361
0.067144744
0.067066
0.06698739
0.06690898
0.06683062
0.0667524
0.06667434
0.06659637
0.06651859
0.06644088
0.06636331
0.06628586
0.06620853
0.06613137
0.066054314
0.065977395
0.065900564
0.06582385
0.06574732
0.06567089
0.06559458
0.06551844
0.065442346
0.065366425
0.06529062
0.06521494
0.06513935
0.06506394
0.06498862
0.06491346
0.06483838
0.06476344
0.06468864
0.06461394
0.06453937
0.0644649
0.064390555
0.06431637
0.0642423
0.064168304
0.06409448
0.0

0.033340506
0.033313468
0.033286445
0.03325946
0.03323251
0.033205606
0.03317874
0.033151872
0.033125076
0.033098344
0.033071566
0.033044882
0.03301821
0.032991596
0.03296499
0.03293845
0.032911934
0.03288543
0.032858986
0.03283256
0.032806166
0.03277983
0.032753512
0.032727223
0.032700974
0.03267477
0.032648563
0.032622427
0.03259632
0.032570258
0.032544207
0.032518193
0.032492235
0.032466277
0.032440376
0.032414515
0.03238867
0.032362863
0.03233712
0.03231137
0.03228566
0.032259993
0.032234345
0.032208752
0.03218318
0.032157637
0.03213213
0.032106653
0.03208124
0.032055795
0.03203045
0.032005083
0.0319798
0.031954527
0.031929277
0.031904064
0.031878892
0.03185373
0.031828634
0.031803556
0.031778503
0.03175349
0.03172851
0.031703547
0.031678643
0.03165376
0.031628914
0.03160408
0.03157931
0.03155452
0.031529807
0.03150511
0.03148041
0.03145581
0.03143123
0.031406626
0.031382095
0.03135761
0.031333108
0.031308666
0.031284265
0.031259872
0.031235546
0.031211212
0.031186923
0.03116266
0.

0.020521617
0.020509614
0.020497655
0.020485695
0.02047374
0.020461807
0.02044987
0.020437948
0.020426048
0.020414157
0.02040228
0.02039043
0.020378571
0.020366728
0.02035487
0.020343058
0.020331252
0.020319473
0.02030767
0.020295909
0.020284161
0.020272402
0.020260675
0.020248953
0.020237248
0.020225544
0.020213846
0.020202167
0.0201905
0.02017885
0.020167211
0.02015558
0.020143954
0.020132355
0.020120764
0.020109178
0.020097595
0.020086039
0.02007448
0.020062935
0.020051418
0.02003989
0.020028401
0.020016896
0.020005431
0.019993965
0.019982487
0.019971035
0.01995961
0.019948186
0.019936785
0.019925358
0.019913968
0.019902594
0.01989123
0.019879863
0.019868512
0.01985719
0.01984586
0.019834539
0.019823242
0.019811947
0.01980067
0.019789394
0.019778147
0.019766893
0.019755661
0.019744445
0.01973323
0.01972203
0.01971084
0.019699655
0.019688481
0.019677328
0.01966618
0.019655047
0.019643914
0.019632787
0.019621698
0.019610588
0.01959952
0.01958845
0.019577393
0.019566342
0.01955531
0.01